In [1]:
# Imports.

from collections import Counter
from sklearn.cluster import KMeans

import colorsys
import cv2
import numpy as np
import os
import pandas as pd

In [2]:
class Face():
    """A class to process an image of a model.
    
    Args:
        image: A 3D numpy array of BGRA pixels.
    """
    
    def __init__(self, image):
        
        # # opencv scales [0, 255]. We want [0, 1].
        self._im = image / 255.0
        
        # Placeholder fields.
        self._df = None
        self._path = None
        self._skin_label = None
        
        
    def make_df(self):
        """Convert the image (3D numpy array) into a pandas dataframe."""
             
        # Reshape into tabular data, taking care to keep the row and column index of each pixel.
        # This could be accomplished with a tangled mess of .ravel() and .vstack(), but numpy commands are illegible.
        (height, width, four) = self._im.shape
        data = []
        for i in range(height):
            for j in range(width):
                [b, g, r, a] = self._im[i, j]
                (h, l, s) = colorsys.rgb_to_hls(r, g, b)
                df_row = [r, g, b, h, l, s, i, j, a]
                data.append(df_row)
        df = pd.DataFrame(data, columns=["r", "g", "b", "h", "l", "s", "row", "col", "a"])
        
        # Remove any transparent pixels.
        df = df.query("a > 0.5")
            
        # Store the dataframe.
        self._df = df
        
        
    def fit_and_predict(self, features, n_clusters):
        """Fit a k-means model to a dataframe of features.

        Args:
            features: list of features (strings) to be used in the k-means model
            n_clusters: int
        """
        
        # Record the name of this model.
        self._path = "".join(features) + str(n_clusters) + "_{}.jpg"

        # Use the data frame of the full image to fit a model.
        df = self._df[features]
        k_means = KMeans(n_clusters=n_clusters)
        k_means.fit(df)
        
        # Calculate and store the labels.
        labels = k_means.predict(df)
        self._df["label"] = labels
        
        
    def calculate_skin_label(self):
        """Determine which label value refers to skin."""
        
        all_labels = set(self._df["label"].tolist())
        darkest_l = 100
        for label in all_labels:
            relevant_rows = self._df[self._df["label"] == label]
            avg_l = relevant_rows["l"].mean()
            # Use the fact that her skin is very dark.
            if avg_l < darkest_l:
                darkest_l = avg_l
                self._skin_label = label
             
            
    def make_and_save_color(self):
        """Create and save her summary skin tone."""
        
        # Grab just the pixels that are skin
        skin_rows = self._df[self._df["label"] == self._skin_label]
        skin_pixels = skin_rows[["b", "g", "r"]] # Curse open cv2 and it's bgr format!! Why!!??

        # Take the median of these skin pixels and coerce into numpy array
        median_pd = skin_pixels.median(axis=0)
        median_np = np.array(median_pd)

        # Save a block of color.
        color_block = np.zeros((100, 100, 3), dtype=np.uint8)
        color_block[:, :] = median_np * 255
        cv2.imwrite(self._path.format("color"), color_block)
    
    
    def make_and_save_image(self):
        """Create and save images to sanity check the model fitting."""
        
        # Store the label for each pixel in order to build images.
        label_dict = {}
        for index, df_row in self._df.iterrows():
            key = (df_row.row, df_row.col)
            value = (df_row.label, df_row.r, df_row.g, df_row.b)
            label_dict[key] = value
        
        # Create a blank image to fill in.
        (height, width, four) = self._im.shape
        im = np.zeros((height, width, 3), dtype=np.uint8)

        # Fill in the pixels.
        for i in range(height):
            for j in range(width):
                key = (i, j)
                
                # This was a background pixel that got removed, so keep it white.
                if key not in label_dict:
                    im[i, j] = [255, 255, 255]
                    continue
                    
                label, r, g, b = label_dict[key]
                
                if label == self._skin_label:
                    im[i, j] = [round(b * 255), round(g * 255), round(r * 255)]
                else:
                    im[i, j] = [255, 240, 180]
        
        cv2.imwrite(self._path.format("skin"), im)

In [3]:
MODELS = [
    (["g", "l"], 3),
    (["r"], 3),
    (["r", "g", "b"], 2),
    (["r", "g", "b"], 3),
]

In [4]:
# Run Florence through the pipeline.

image_path = "removed.png"
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
face = Face(image)
face.make_df()
    
for (features, n_clusters) in MODELS:
    print("features:", features, "\nn_clusters:", n_clusters, "\n")
    face.fit_and_predict(features, n_clusters)
    face.calculate_skin_label()
    face.make_and_save_color()
    face.make_and_save_image()

features: ['g', 'l'] 
n_clusters: 3 

features: ['r'] 
n_clusters: 3 

features: ['r', 'g', 'b'] 
n_clusters: 2 

features: ['r', 'g', 'b'] 
n_clusters: 3 

